<a href="https://colab.research.google.com/github/FelipeNemo/email-validation-guide/blob/public-e-mail-verificador/emailvalidation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Sistema de Validação de Email para prevenção de fraude**

**Autor:** Felipe Augusto Batista Mendes dos Santos



Este sistema atende à necessidade de validação de e-mails para aumentar a eficiência e a segurança das transações online, utilizando um pipeline de processamento. Ele visa usar o status de classificação de validade para a tomada de decisões.

##  **Input:** Dados estruturados

 ##  **Output:** Status de classificação de e-mail

# **Verificação de Email**
Verificação de um endereço de email existente e ativo, ou seja, válido. Um email válido pode receber mensagens de outros remetentes. São responsáveis por executar uma etapa do **PROCESSO DE VERIFICAÇÃO**, como por exemplo:
* Verificações de sintaxe
* Domínio
* Registro MX
* Catch-all
* Ping de endereço de e-mail

# **Validação de email**
É o resultado final do processo de verificação de email. Às vezes, a validação também implica na verificação de qualquer informação adicional que vem com o e-mail do lead ou na verificação da origem desse lead.

* “Não verificado” ⭕ - antes do processo de verificação.

* “Válido” 🟢 - significa que um e-mail está correto e existe.

* “Inválido” 🟡 -  significa que esse endereço de e-mail pode receber mensagens, mas devido a uma configuração em todo o domínio, não é possível determinar a sua validade real, assim ele é marcado como Não verificável. Esses e-mails também são conhecidos como e-mails “pegam tudo” ou “aceitam tudo”.

* “Não verificável” 🔴 – significa que um endereço de e-mail não existe ou não está mais ativo.

*  “Não é possível encontrar o e-mail” ⚪ - clientes potenciais com informações pessoais, como fotos ou experiência profissional etc., no entanto, não há endereço de e-mail.

# **Explicação da regra de negócio:**

## **Não verificado ⭕ :**
– 1. Ainda não entrou em nenhum processo de validação

## **Não verificável 🔴 :**
– 1. O e-mail falha na verificação de sintaxe (etapa 2).

– 2. O domínio não possui um registro A válido (etapa 3).

– 3. O domínio não possui registros MX válidos (etapa 4).

– 4. O domínio não responde à verificação SMTP (etapa 5).

## **Não é possível encontrar o e-mail ⚪ :**
– 1. O e-mail não contém um símbolo "@" ou outras informações importantes estão ausentes, impedindo a verificação de sintaxe (etapa2).         
## **Inválido 🟡 :**
– 1. O e-mail pode receber mensagens, mas devido a uma configuração em todo o domínio (catch-all), não é possível determinar sua validade real (etapa 4).

## **Válido 🟢 :**
– 1. Sintaxe correta (etapa 2).

– 2. Domínio existente possui registro A válido (etapa 3).

– 3. Domínio possui registros MX válidos (etapa 4).

– 4. Domínio não está configurado como catch-all (etapa 4).

– 5. Domínio responde à verificação SMTP (etapa 5).

# **Para que verificar emails?**

1.    Melhora as taxas de entrega de e-mails
2.    Economia de tempo e despesas com e-mail marketing
3.    Mantém a reputação do remetente elevada
4.    Melhora a comunicação com os leads
5.    Sinaliza possiveis ataques de fraude ao seu e-commerce










# **Como funciona um verificador de e-mail:**
1.   Verificação de sintaxe
2.   Verificação de jargão
3.   Verificação de existência de domínio
4.   Verificação de registro MX
5.   Verificação de domínio catch-all
6.   Autenticação SMTP (ping de endereço de e-mail)
7.   Verificação de freemail




# **Install:**

In [2]:
!pip install dnspython requests


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 5.9 MB/s eta 0:00:00


# **Importações:**

In [3]:
import csv
import dns.resolver #módulo da lib dnspython
import smtplib
import requests
import pandas as pd

# **1. Verificação de sintaxe:**
#### **Ferramentas:**  

*   Python

#### **Correspondência de padrão de email:**

> < local-part@domain >

1. **local-part** -  Pode ter até 64 octetos de comprimento, disposição de pontos e caracteres especiais.

2. **@** - Deve conter apenas 1 elemento "@"em todo o e-mail

3. **domain** -  Pode ter no máximo 255 octetos e deve conter apenas 1 "."



### **1.1 Local-part:**
#### **Ferramentas:**

*   dnspython
 **Disposição de bytes:**
*  64 octetos de comprimento

#### **Disposição geral de pontos:**

 Não é permitido que seja o primeiro ou o último caractere, e que também apareça consecutivamente

* **Caracteres especiais:**

> "() , : ; < > @ [ \ ]

  -  São permitidos com restrições(somente dentro de uma string entre aspas e nessa string entre aspas, qualquer barra invertida ou aspas duplas deve ser precedida uma vez por uma barra invertida)
   
   Exemplo:
   
   john.smith(comment)@example.come e (comment)john.smith@example.com são ambos equivalentes a john.smith@example.com

   

In [4]:
CARACTERES_ESPECIAIS = {"(", ")", ",", ":", ";", "<", ">", "@", "[", "]"}

In [5]:
def local_sintax(local):
    # Função de verificação da parte local
    if len(local) == 0 or len(local) > 64:
        return False
    if local[0] == '.' or local[-1] == '.':
        return False
    if '..' in local:
        return False
    for char in local:
        if char in CARACTERES_ESPECIAIS:
            return False
    return True

### **1.2 Domain**
#### **Ferramentas:**  

*   Python

#### **Disposição de bytes:**
* 255 octetos de comprimento

**Disposição geral de pontos:**
  
* Deve conter apenas 1 "." :

In [6]:
def domain_sintax(domain):
    # Função de verificação da parte do domínio
    if len(domain) == 0 or len(domain) > 255:
        return False
    if '.' not in domain:
        return False
    if domain[0] == '-' or domain[-1] == '-':
        return False
    for char in domain:
        if char in CARACTERES_ESPECIAIS:
            return False
    # Verifica cada parte do domínio
    labels = domain.split('.')
    for label in labels:
        if len(label) == 0 or len(label) > 63:
            return False
    return True

### **Conclusão:**

In [7]:
def verificador_sintax(email):
    # Verifica se contém exatamente um "@"
    if email.count('@') != 1:
        print("Falhou: não contém exatamente um '@'")
        return False

    local, domain = email.split('@')

    # Verificação da parte local
    if not local_sintax(local):
        print("Falhou: verificação da parte local")
        return False

    # Verificação da parte do domínio
    if not domain_sintax(domain):
        print("Falhou: verificação da parte do domínio")
        return False

    return True

# **2. Verificação de domínio:**
#### **Ferramentas:**

*   dnspython

#### **O que é o registro A ?**

Um Registro A(Address Record) é um tipo de registro DNS que mapeia um nome de domínio para um endereço IP único (IPv4).

 Permite que um domínio(gmail.com) seja associado a um endereço IP (147.0.2.1), que é necessário para que os navegadores da web possam encontrar o servidor correspondente na rede.


In [8]:
def check_domain_exists(domain):
    try:
        # Verifica registros DNS
        dns.resolver.resolve(domain, 'A')
        return True
    except (dns.resolver.NXDOMAIN, dns.resolver.NoAnswer, dns.resolver.Timeout):
        return False



In [9]:
def check_domain_active(domain):
    try:
        # Faz uma requisição HTTP ao domínio
        response = requests.get(f"http://{domain}", timeout=10)
        return response.status_code == 200
    except requests.RequestException:
        return False

# **3. Verificação do registro MX:**
#### **Ferramentas:**

*   dnspython

#### **O que é o registro MX?**

Um MX record (Mail Exchange record) é um tipo de registro DNS (Domain Name System) que especifica qual servidor de e-mail é responsável por receber e-mails para um domínio específico.

 Ele funciona verificando se a entrada de troca de correio listada no endereço de e-mail é um nome de domínio real, registrado e hospedado, que pode ser acessado online.


#### **Erros tratados:**

*NoAnswer :* Não há registros MX para o domínio.

*NXDOMAIN :* O domínio não existe.

*Timeout :* Timeout ao consultar o domínio.

*NoNameservers :* Servidores de nomes não encontrados para o domínio.

*YXDOMAIN :* Nome do domínio muito longo.

*NoRootSOA :* O domínio não possui uma SOA válida.


In [10]:
def get_mx_records(domain):
    try:
        answers = dns.resolver.resolve(domain, 'MX')
        mx_records = sorted([(rdata.preference, str(rdata.exchange)) for rdata in answers], key=lambda x: x[0])
        return [record[1] for record in mx_records]
    except dns.resolver.NoAnswer:
        print(f"Não há registros MX para o domínio: {domain}")
        return None
    except dns.resolver.NXDOMAIN:
        print(f"O domínio {domain} não existe.")
        return None
    except dns.resolver.Timeout:
        print(f"Timeout ao consultar o domínio: {domain}")
        return None
    except dns.resolver.NoNameservers:
        print(f"Os servidores de nomes não puderam ser encontrados para o domínio: {domain}")
        return None
    except dns.resolver.YXDOMAIN:
        print(f"Nome do domínio muito longo para: {domain}")
        return None
    except dns.resolver.NoRootSOA:
        print(f"Domínio {domain} não possui uma SOA válida.")
        return None
    except dns.exception.DNSException as e:
        print(f"Erro ao consultar os registros MX do domínio {domain}: {e}")
        return None



# **4.Verificação catch-all:**
#### **Ferramentas:**

*   dnspython
*   smtplib

#### **O que é um domínio "catch-all"?**

É quando o email está configurado para aceitar emails enviados para qualquer endereço no domínio, mesmo que o endereço específico não exista.

Assim, o domínio reconhece a diferença entre um email real (renata.pereira@gmail.com) e um email aleatório/inexistente naquele domínio (renDPokfçsnoppereira@gmail.com).


In [11]:
def check_catch_all(domain):
    mx_records = get_mx_records(domain)
    if not mx_records:
        print(f"Não foi possível encontrar registros MX para o domínio: {domain}")
        return False
    # E-mail Teste
    test_email = f"catchall-test-{domain}@{domain}"
    sender_email = "test@example.com"

    for mx in mx_records:
        try:
            server = smtplib.SMTP(mx)
            server.set_debuglevel(0)  # Defina para 1 se quiser ver a comunicação SMTP
            server.ehlo_or_helo_if_needed()
            server.mail(sender_email)
            code, message = server.rcpt(test_email)
            server.quit()

            if code == 250:
                print(f"O domínio {domain} possui um catch-all configurado.")
                return True
        except smtplib.SMTPServerDisconnected:
            pass
        except smtplib.SMTPConnectError:
            pass
        except smtplib.SMTPRecipientsRefused:
            pass
        except Exception as e:
            print(f"Erro ao verificar o domínio {domain}: {e}")
            return False

    print(f"O domínio {domain} não possui um catch-all configurado.")
    return False


# **5. Ping do endereço de e-mail:**
#### **Ferramentas:**

*   dnspython
*   smtplib

#### **O que é o SMTP?**

Utilizamos a verificação do SMTP (Simple Mail Transfer Protocol), que é o método padrão para envio de emails pela internet, para enviar um ping/email-de-teste destinado ao endereço de e-mail, verificando se ele é capaz de receber mensagens e reagir a elas.

In [12]:

def ping_email_address(email):
    domain = email.split('@')[1]
    mx_records = get_mx_records(domain)
    if not mx_records:
        print(f"Não foi possível encontrar registros MX para o domínio: {domain}")
        return False

    test_email = email
    sender_email = "test@example.com"  # Pode ser qualquer endereço válido

    for mx in mx_records:
        try:
            # Conectar ao servidor SMTP
            server = smtplib.SMTP(mx, timeout=10)
            server.set_debuglevel(0)  # Defina para 1 se quiser ver a comunicação SMTP
            server.helo()
            server.mail(sender_email)
            code, message = server.rcpt(test_email)
            server.quit()

            if code == 250:
                print(f"O e-mail {email} parece ser válido.")
                return True
            else:
                print(f"O e-mail {email} não parece ser válido. Código de resposta: {code}")
                return False
        except smtplib.SMTPConnectError:
            print(f"Erro ao conectar com o servidor de e-mail {mx}.")
        except Exception as e:
            print(f"Erro ao verificar o e-mail {email}: {e}")

    print(f"Não foi possível verificar o e-mail {email}.")
    return False


# **Atribuindo a regra de negócio:**


In [13]:

def verificar_email(email):
    status = "Não verificado ⭕"

    if not verificador_sintax(email):
        status = "Não verificável 🔴"
        return status

    domain = email.split('@')[1]

    if not check_domain_exists(domain):
        status = "Não verificável 🔴"
        return status

    if not check_domain_active(domain):
        status = "Não verificável 🔴"
        return status

    mx_records = get_mx_records(domain)
    if not mx_records:
        status = "Não verificável 🔴"
        return status

    if check_catch_all(domain):
        status = "Inválido 🟡"
        return status

    if not ping_email_address(email):
        status = "Não verificável 🔴"
        return status

    status = "Válido 🟢"
    return status


# **Leitura de dados estruturados:**

* Como exemplo, gerei e-mails falsos, por isso a validação veio com o status 'Connection unexpectedly closed'. Para testar, substitua por um arquivo com e-mails reais e faça a leitura nas células abaixo utilizando o Pandas.

In [14]:
planilha = pd.read_csv('./dados_aleatorios.csv')
planilha.head()

,ID,Nome,Códigos de rastreio,Email,Estado
0,84015184,Lucas Oliveira,NVA5SR7GC4OE,lucas.oliveira@uol.com.br,ES
1,83494978,Renata Pereira,C5WRXE33NSNK,renata.pereira@gmail.com,PR
2,36146937,Renata Costa,MEQ365ZO36PT,renata.costa@dominio-muito-longo-de-email.com.br,PR
3,91738901,Felipe Silva,YFHEK9PJD0XT,felipe.silva@outlook.com,ES
4,95558529,Felipe Souza,FNXN6YYQJ9GW,felipe.souza@outlook.com,SC


In [17]:
email_coluna = planilha['Email']
print(email_coluna.head())

0                           lucas.oliveira@uol.com.br
1                            renata.pereira@gmail.com
2    renata.costa@dominio-muito-longo-de-email.com.br
3                            felipe.silva@outlook.com
4                            felipe.souza@outlook.com
Name: Email, dtype: object


In [18]:
emails = email_coluna.tolist()
print(emails)

['lucas.oliveira@uol.com.br', 'renata.pereira@gmail.com', 'renata.costa@dominio-muito-longo-de-email.com.br', 'felipe.silva@outlook.com', 'felipe.souza@outlook.com', 'carla.araujo@outlook.com', 'marcos.araujo@gmail.com', 'renata.costa@dominio-muito-longo-de-email.com.br', 'lucas.almeida@gmail.com', 'ana.pereira@uol.com.br']


In [19]:
# Função para verificar a lista de e-mails e salvar o resultado em um CSV
def verificar_emails_lista(emails, csv_file):
    resultados = []

    for email in emails:
        status = verificar_email(email)
        resultados.append({"email": email, "status": status})

    with open(csv_file, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=["email", "status"])
        writer.writeheader()
        writer.writerows(resultados)

# Nome do arquivo CSV para salvar os resultados
csv_file = "validacao_emails.csv"

# Verificar os e-mails e salvar os resultados
verificar_emails_lista(emails, csv_file)

O domínio uol.com.br não possui um catch-all configurado.
Erro ao verificar o e-mail lucas.oliveira@uol.com.br: Connection unexpectedly closed
Não foi possível verificar o e-mail lucas.oliveira@uol.com.br.
O domínio gmail.com não possui um catch-all configurado.
O e-mail renata.pereira@gmail.com parece ser válido.
O domínio outlook.com não possui um catch-all configurado.
Erro ao verificar o e-mail felipe.silva@outlook.com: Connection unexpectedly closed
Não foi possível verificar o e-mail felipe.silva@outlook.com.
O domínio outlook.com não possui um catch-all configurado.
Erro ao verificar o e-mail felipe.souza@outlook.com: Connection unexpectedly closed
Não foi possível verificar o e-mail felipe.souza@outlook.com.
O domínio outlook.com não possui um catch-all configurado.
Erro ao verificar o e-mail carla.araujo@outlook.com: Connection unexpectedly closed
Não foi possível verificar o e-mail carla.araujo@outlook.com.
O domínio gmail.com não possui um catch-all configurado.
O e-mail mar

# **Documentações:**

*   pandas: https://pandas.pydata.org/docs/
*   dsnpython: https://dnspython.readthedocs.io/en/latest/
*   requests: https://requests.readthedocs.io/projects/pt/pt-br/latest/user/quickstart.html
*  csv: https://docs.python.org/3/library/csv.html
* smtplib: https://docs.python.org/3/library/smtplib.html

# **Referências:**
* Artigo base de referência:

 https://snov.io/blog/br/verificacao-de-email-explicada/

* API de referência:

 https://hunter.io/api-documentation/v2#email-verifier


*   Plataforma de validação de email que inspirou o projeto:

 https://snov.io/br/verificador-de-emails
*  Entendendo DNS:

 https://support.google.com/a/answer/48090?hl=pt-BR#:~:text=voltar%20ao%20in%C3%ADcio-,Registro%20MX,de%20e%2Dmail%20do%20Google.
*   Status de email:

 https://snov.io/knowledgebase/email-statuses/

*   Definições de leads:

 https://fleeg.com/blog/inbound-marketing/leads/?origin=fleeg_snovio_link_backlink&utm_source=Snovio&utm_medium=link&utm_campaign=backlink
*   Verificação sintaxe:

 https://en.wikipedia.org/wiki/Email_address#Local-part

 https://en.wikipedia.org/wiki/Octet_(computing)

 https://www.youtube.com/watch?v=WkNBhYMUXwI
*   Envio de email SMTP:

 https://www.youtube.com/watch?v=umvzsQLZYD4
*   Verificação TLD/MX record/Sintaxe:

 https://www.youtube.com/watch?v=Bj61jJo2Mvs

 https://gcore.com/learning/dns-mx-record-explained/



